In [4]:

#  Script Author:  Bill Katzenmeyer
#  How to Set up Python and PyDSS https://www.linkedin.com/posts/william-katzenmeyer_hec-hec-python-activity-7056763454753275904-1RcT
#
#  Please use HEC-DSSVue to populate the search terms below (River Station and Search Word) if you get blank results
#
#  For Best Results: 
#  1. Use one DSS file per run 
#  2. One set of results per file
#  3. All results using same Date/Time Range 
#
#  This can be achieved by deleting any DSS results files in the RAS folder before running
#  (As RAS will append DSS results to an existing file)
#  There is an option to use the plan name for the DSS file in the Unsteady window 
#
#  Example DSS files and Gauge_Data.csv are provided
#
#  This script was built using ChatGPT4, and is a starting point for DSS data postprocessing
#


import os
import pandas as pd
import numpy as np
from pydsstools.heclib.dss import HecDss


# BEGIN User Inputs ---------------------------------------------------

folder_path = "C:/DSS FILE OUTPUTS/DSS"  
# Folder path where DSS files are located

river_stations = ["90128", "117188", "110659", "7458"]  
# List of river station numbers to filter pathnames

search_word = "Harvey"  
# Search word to filter pathnames

pathname_pattern = "/*/*/*/*/*/*/"  
# Pathname pattern to filter pathnames

start_date = "26AUG2017 00:00:00"  
# Start date for reading time series data

end_date = "14SEP2017 00:00:00"  
# End date for reading time series data

csv_path = "Time_Series_Output.csv"

# END User Inputs ---------------------------------------------------


def check_identical_values(df, new_series):
    for column in df.columns:
        if np.array_equal(df[column].head(100).values, new_series.head(100).values):
            return True
    return False

df = pd.DataFrame()
for file in os.listdir(folder_path):
    if file.endswith(".dss"):
        with HecDss.Open(os.path.join(folder_path, file)) as fid:
            path_list = fid.getPathnameList(pathname_pattern, sort=1)
            filtered_path_list = [
                path for path in path_list
                if any(station in path for station in river_stations)
                and search_word in path
                and ("FLOW" in path or "STAGE" in path)
                and "FLOW-CUM" not in path
            ]
            for pathname in filtered_path_list:
                ts = fid.read_ts(pathname, window=(start_date, end_date), trim_missing=True)
                if ts is not None:
                    times, values = np.array(ts.pytimes), ts.values
                    header = f"{file} {pathname}"
                    temp_df = pd.DataFrame({header: pd.Series(values[~ts.nodata], index=times[~ts.nodata])})
                    if not check_identical_values(df, temp_df[header]):
                        df = pd.concat([df, temp_df], axis=1)

df.to_csv(os.path.join(folder_path, csv_path), index_label="Date")
